In [1]:
!pip install pillow

In [2]:
!pip install opencv-python

In [3]:
# Basic Imports
import numpy as np
import pandas as pd

# Project Imports
import os
import cv2
from PIL import Image

In [4]:
# Path & Variables
CUR_DIR = os.getcwd()
DATA_DIR = os.path.join(CUR_DIR, 'data')

RAW_DATA = os.path.join(DATA_DIR, 'raw_data')
DATA_CLEAN = os.path.join(DATA_DIR, 'data_clean')

DIM = (128, 128)

In [5]:
# Check & create target dir
if not os.path.exists(DATA_CLEAN):
    os.makedirs(DATA_CLEAN)

In [6]:
# Convert & Process image into npy

for file in os.listdir(RAW_DATA):
    if file.endswith(".jpg"):
        img = cv2.imread(os.path.join(RAW_DATA, file))
        resize_img = cv2.resize(img, DIM)
        # img_array = np.array(img)
        file_name = os.path.splitext(file)[0]
        np.save(os.path.join(DATA_CLEAN, file_name), resize_img)

FileNotFoundError: [Errno 2] No such file or directory: '/home/remy/code/Lalucarne51/mammo-analysis/notebooks/data/raw_data'

In [ ]:
# Adding the target
metadata = pd.read_excel(os.path.join(DATA_DIR, 'metadata.xlsx'))
img = np.load(os.path.join(DATA_CLEAN, os.listdir(DATA_CLEAN)[0]))

metadata_sample = metadata[['cancer', 'image_id']]
metadata_sample.head(2)

,cancer,image_id
0,0,462822612
1,0,1459541791


In [ ]:
# Create CSV for the sample (array & Id)
image_id = []
img_arrays = []

for img in os.listdir(DATA_CLEAN):
    image_id.append(np.int64(int(os.path.splitext(img)[0])))
    img_arrays.append(np.load(os.path.join(DATA_CLEAN, img)))

img_csv = pd.DataFrame(dict(
    image_id=image_id,
    image_array=img_arrays
))


In [ ]:
final_csv = img_csv.merge(metadata_sample, on= 'image_id', how='left')
final_csv.shape

(1667, 3)

In [ ]:
final_csv.head()

,image_id,image_array,cancer
0,257647980,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
1,879478870,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",0
2,1695802156,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",0
3,1232990271,"[[[161, 161, 161], [157, 157, 157], [162, 162,...",0
4,248287572,"[[[204, 204, 204], [204, 204, 204], [204, 204,...",0


In [ ]:
# Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def initialize_model(X):

    model = Sequential()

    model.add(Conv2D(16, (4, 4), input_shape = (X[0].shape[0], X[0].shape[1], X[0].shape[2]), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units = 10, activation = 'relu'))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                 metrics =['accuracy', 'Recall'])

    return model

In [ ]:
# Get Feature & target
X = final_csv.image_array
y = final_csv.cancer
X.shape, y.shape

((1667,), (1667,))

In [ ]:
# Init Model
model = initialize_model(X)
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 253, 253, 16)      784       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 126, 126, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 124, 124, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 62, 62, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 30, 30, 64)       

In [ ]:
# Convert


pandas.core.series.Series

In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# One Hot Encode our Target for TensorFlow processing
from tensorflow.keras.utils import to_categorical
y_cat = to_categorical(y, num_classes=2)
y.shape, y_cat.shape

((1667,), (1667, 2))

In [ ]:

history = model.fit(X,
                    y_cat,
                    validation_split = 0.2,
                    batch_size = 32,
                    epochs = 5,
                    verbose = 1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).